<a href="https://colab.research.google.com/github/aniparnagian/Divine-Comedy-Topic-Modeling/blob/main/Dante's_Inferno_Topic_Modeling_IPHS_300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LDA Topic Modeling with Gensim

Heavily modified and expanded based upon Gensim tutorial

By Jon Chun
28 Mar 2022 Updated

# Setup and Configuration

In [ ]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 25.2 MB/s eta 0:00:00


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 15.0 MB/s eta 0:00:00


## Configure Jupyter Notebook

In [ ]:
## Configure Jupyter Notebook

# Ignore warnings

import warnings
warnings.filterwarnings('ignore')

# Enable multiple outputs from one code cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
from IPython.display import Image
from ipywidgets import widgets, interactive

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## [INPUT] Connect Google gDrive to this Jupyter Notebook

In [ ]:
# [INPUT REQUIRED]: Authorize access to Google gDrive via popup windows

# Connect this Notebook to your permanent Google Drive
#   so all generated output is saved to permanent storage there
"""
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("Attempting to attach your Google gDrive to this Colab Jupyter Notebook")
  drive.mount('/gdrive')
else:
  print("Your Google gDrive is attached to this Colab Jupyter Notebook")
"""

'\ntry:\n  from google.colab import drive\n  IN_COLAB=True\nexcept:\n  IN_COLAB=False\n\nif IN_COLAB:\n  print("Attempting to attach your Google gDrive to this Colab Jupyter Notebook")\n  drive.mount(\'/gdrive\')\nelse:\n  print("Your Google gDrive is attached to this Colab Jupyter Notebook")\n'

# Data

You have (2) ways to get data in this tutorial, but if you're following
this tutorial just to learn about LDA I encourage you to consider picking a
corpus on a subject that you are familiar with. Qualitatively evaluating the
output of an LDA model is challenging and can require you to understand the
subject matter of your corpus (depending on your goal with the model).

Reference to Compare::

    The NeurIPS corpus contains 1740 documents, and not particularly long ones.

    `website <http://www.cs.nyu.edu/~roweis/data.html>`
    
    So keep in mind that this tutorial is not geared towards efficiency, and be
    careful before applying the code to a large dataset.


In [ ]:
!ls -al

total 908
drwxr-xr-x 1 root root   4096 Mar  3 02:15  .
drwxr-xr-x 1 root root   4096 Mar  3 02:13  ..
drwxr-xr-x 4 root root   4096 Mar  1 14:33  .config
-rw-r--r-- 1 root root 912267 Mar  3 02:15 'Divine Comedy.txt'
drwxr-xr-x 1 root root   4096 Mar  1 14:34  sample_data


In [ ]:
# If you have several files, use this to concatenate into one file
# !cat (RegEx pattern) > one_combined_file.txt

!cat R*.txt > file_all.txt

cat: 'R*.txt': No such file or directory


## Option (a): Use wget Command: Gdrive

In [ ]:
# Test to get the 7 Harry Potter novels as one combined zip file via URL

!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=15J8QLbyOh-6B5RlfjrQZ5CyN0fwsH0T_' -O potter7.zip

--2023-03-01 16:32:09--  https://docs.google.com/uc?export=download&id=15J8QLbyOh-6B5RlfjrQZ5CyN0fwsH0T_
Resolving docs.google.com (docs.google.com)... 74.125.201.113, 74.125.201.138, 74.125.201.101, ...
Connecting to docs.google.com (docs.google.com)|74.125.201.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8p9s9afcfgjlma1kfvms3ivqg72aj79c/1677688275000/14430240466678867548/*/15J8QLbyOh-6B5RlfjrQZ5CyN0fwsH0T_?e=download&uuid=3646537f-99ec-4cf8-9705-0294c78340ed [following]
--2023-03-01 16:32:10--  https://doc-00-70-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8p9s9afcfgjlma1kfvms3ivqg72aj79c/1677688275000/14430240466678867548/*/15J8QLbyOh-6B5RlfjrQZ5CyN0fwsH0T_?e=download&uuid=3646537f-99ec-4cf8-9705-0294c78340ed
Resolving doc-00-70-docs.googleusercontent.com (doc-00-70-docs.googleusercontent.com)... 173.194.192.132, 2607:f8b0

In [ ]:
# Verify

!ls -al

total 2460
drwxr-xr-x 1 root root    4096 Mar  1 16:32 .
drwxr-xr-x 1 root root    4096 Mar  1 16:29 ..
drwxr-xr-x 4 root root    4096 Feb 27 14:36 .config
-rw-r--r-- 1 root root       0 Mar  1 16:32 file_all.txt
-rw-r--r-- 1 root root 2499084 Mar  1 16:32 potter7.zip
drwxr-xr-x 1 root root    4096 Feb 27 14:36 sample_data


In [ ]:
# If you have a zip archive, you need to unzip it

!unzip potter7.zip

Archive:  potter7.zip
  inflating: Rowling, J.K. - HP 1 - Harry Potter and the Sorcerer's Stone.txt  
  inflating: Rowling, J.K. - HP 2 - Harry Potter and the Chamber of Secrets.txt  
  inflating: Rowling, J.K. - HP 3 - Harry Potter and the Prisoner of Azkaban.txt  
  inflating: Rowling, J.K. - HP 4 - Harry Potter and the Goblet of Fire.txt  
  inflating: Rowling, J.K. - HP 5 - Harry Potter and the Order of the Phoenix.txt  
  inflating: Rowling, J.K. - HP 6 - Harry Potter and the Half-Blood Prince.txt  
  inflating: Rowling, J.K. - HP 7 - Harry Potter and the Deathly Hallows.txt  


In [ ]:
# concatenate all individual files into one text file

!cat Rowling* > potter_all.txt

In [ ]:
!ls -al

total 15520
drwxr-xr-x 1 root root    4096 Mar  1 16:32  .
drwxr-xr-x 1 root root    4096 Mar  1 16:29  ..
drwxr-xr-x 4 root root    4096 Feb 27 14:36  .config
-rw-r--r-- 1 root root       0 Mar  1 16:32  file_all.txt
-rw-r--r-- 1 root root 2499084 Mar  1 16:32  potter7.zip
-rw-r--r-- 1 root root 6681950 Mar  1 16:32  potter_all.txt
-rw-r--r-- 1 root root  450409 Dec 21  2020 "Rowling, J.K. - HP 1 - Harry Potter and the Sorcerer's Stone.txt"
-rw-r--r-- 1 root root  538387 Dec 21  2020 'Rowling, J.K. - HP 2 - Harry Potter and the Chamber of Secrets.txt'
-rw-r--r-- 1 root root  626245 Dec 21  2020 'Rowling, J.K. - HP 3 - Harry Potter and the Prisoner of Azkaban.txt'
-rw-r--r-- 1 root root 1125194 Dec 21  2020 'Rowling, J.K. - HP 4 - Harry Potter and the Goblet of Fire.txt'
-rw-r--r-- 1 root root 1627990 Dec 13  2020 'Rowling, J.K. - HP 5 - Harry Potter and the Order of the Phoenix.txt'
-rw-r--r-- 1 root root 1072750 Dec 13  2020 'Rowling, J.K. - HP 6 - Harry Potter and the Half-Blood Pri

In [ ]:
# Verify content

!head -n 200 potter_all.txt

Harry Potter and the Sorcerer's Stone
by J.K. Rowling

CHAPTER ONE

THE BOY WHO LIVED

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.

Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.

The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about 

In [ ]:
# set the variable 'textfile_name' to the name of your 

textfile_name = 'potter_all.txt'

## Option (b): Put Text Datafile in your GDrive project directory

If you have a Text Datafile ready to use, just copy it into your GDrive folder that is the project directory for this LDA exercise (listed below)

In [ ]:
print(f'Put your Textfile (named: textfile_name) in the current project directory :\n')
!pwd

Put your Textfile (named: textfile_name) in the current project directory :

/content


In [ ]:
print(f'Set var textfile_name = "name of uploaded file.txt"')
textfile_name = 'name of uploaded file.txt'

Set var textfile_name = "name of uploaded file.txt"


**[SKIP] to next Section [LDA MODEL]**

## Option (c): Use wget Command with Gutenberg.org

If you don't have a Textfile ready, let's just grab one to be able to continue learning about Gensim LDA.

[wget] is a Unix utility that allows you to wEB GET and download most any publically available file on the Internet given the URL to it.

Option (b)(i): If you have a URL to an open, freely available text document enter it in the next code cell below

Option (b)(ii): If you don't have a URL, 

* goto https://www.gutenberg.org/

* search for a longish novel (e.g. 'middlemarch' at https://www.gutenberg.org/ebooks/145)

* Click on the 'Plain Text UTF-8' version to goto the plain text version of the novel, copy the URL and paste it in the next code cell below

In [ ]:
# Assign the variable url to your link address
#  (e.g. url = 'https://www.gutenberg.org/ebooks/145')

url_text = 'https://www.gutenberg.org/files/145/145-0.txt'

!wget $url_text

--2023-03-01 16:27:46--  https://www.gutenberg.org/files/145/145-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1865506 (1.8M) [text/plain]
Saving to: ‘145-0.txt’

145-0.txt           100%[===================>]   1.78M  10.4MB/s    in 0.2s    

2023-03-01 16:27:51 (10.4 MB/s) - ‘145-0.txt’ saved [1865506/1865506]



In [ ]:
# What filename was just downloaded
#   (time sorted, so at top of list below)

!ls -altr *145*

-rw-r--r-- 1 root root 1865506 Feb 21 21:40 145-0.txt


In [ ]:
# [OPTIONAL]: Make your Textdata filename more user-friendly

textfile_downloaded = '145-0.txt'
textfile_name = 'middlemarch.txt'

!mv $textfile_downloaded $textfile_name

!ls -altr middle*

-rw-r--r-- 1 root root 1865506 Feb 21 21:40 middlemarch.txt


In [ ]:
!head middlemarch.txt

﻿The Project Gutenberg eBook of Middlemarch, by George Eliot

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.



In [ ]:
textfile_name = 'middlemarch.txt'


# LDA Model

Introduces Gensim's LDA model and demonstrates its use on the NIPS corpus.


The purpose of this tutorial is to demonstrate how to train and tune an LDA model.

In this tutorial we will:

* Load input data.
* Pre-process that data.
* Transform documents into bag-of-words vectors.
* Train an LDA model.

This tutorial will **not**:

* Explain how Latent Dirichlet Allocation works
* Explain how the LDA model performs inference
* Teach you all the parameters and options for Gensim's LDA implementation

If you are not familiar with the LDA model or how to use it in Gensim, I (Olavur Mortensen)
suggest you read up on that before continuing with this tutorial. Basic
understanding of the LDA model should suffice. Examples:

* `Introduction to Latent Dirichlet Allocation <http://blog.echen.me/2011/08/22/introduction-to-latent-dirichlet-allocation>`_
* Gensim tutorial: `sphx_glr_auto_examples_core_run_topics_and_transformations.py`
* Gensim's LDA model API docs: :py:class:`gensim.models.LdaModel`

I would also encourage you to consider each step when applying the model to
your data, instead of just blindly applying my solution. The different steps
will depend on your data and possibly your goal with the model.





## Read Textfile

In [ ]:
!ls -al

total 908
drwxr-xr-x 1 root root   4096 Mar  3 02:16  .
drwxr-xr-x 1 root root   4096 Mar  3 02:13  ..
drwxr-xr-x 4 root root   4096 Mar  1 14:33  .config
-rw-r--r-- 1 root root 912267 Mar  3 02:15 'Divine Comedy.txt'
-rw-r--r-- 1 root root      0 Mar  3 02:16  file_all.txt
drwxr-xr-x 1 root root   4096 Mar  1 14:34  sample_data


In [ ]:
textfile_name = 'Divine Comedy.txt'
textfile_name

'Divine Comedy.txt'

In [ ]:
with open(textfile_name, 'r', encoding='ascii', errors='ignore') as f:
    book_all_str = f.read()

type(book_all_str)

str

In [ ]:
# Get char count
len(book_all_str)

861482

In [ ]:
book_all_str[:1000]

'The Project Gutenberg eBook of The Divine Comedy, by Dante Alighieri\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org. If you are not located in the United States, you\nwill have to check the laws of the country where you are located before\nusing this eBook.\n\nTitle: The Divine Comedy\n\nAuthor: Dante Alighieri\n\nTranslator: Henry Francis Cary\n\nRelease Date: August, 1997 [eBook #1008]\n[Most recently updated: July 4, 2022]\n\nLanguage: English\n\n\nProduced by: Judith Smith and Natalie Salter\n\n*** START OF THE PROJECT GUTENBERG EBOOK THE DIVINE COMEDY ***\n\n\n\n\nTHE DIVINE COMEDY OF DANTE ALIGHIERI\n\nTranslated by\nTHE REV. H. F. CARY, M.A.\n\n\n\n\nContents\n\n HELL\n CANTO I.\n CANTO II.\n CANTO III.\n CANTO IV.\n CA

## Split the Book/Corpus into Paragraphs/Documents

In [ ]:
# Split into paragraphs
book_parags_ls = book_all_str.split('\n\n')

# Replace stray/embedded /n with a space
book_parags_clean_ls = []
for aparag in book_parags_ls:
  book_parags_clean_ls.append(aparag.replace('\n',' '))

book_parags_ls = book_parags_clean_ls
book_parags_ls[:5]

['The Project Gutenberg eBook of The Divine Comedy, by Dante Alighieri',
 'This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.',
 'Title: The Divine Comedy',
 'Author: Dante Alighieri',
 'Translator: Henry Francis Cary']

In [ ]:
# Get paragraph count
len(book_parags_ls)

2968

In [ ]:
# Verify first paragraph (after title and chapter headings)

book_parags_ls[3]
print('\n')
# Paragraph char count
len(book_parags_ls[3])

'Author: Dante Alighieri'

23

In [ ]:
# Delete any paragraphs shorter than MIN_LEN_PARAG

MIN_LEN_PARAG = 5
MIN_LEN_DOC = 1000

# Delete any paragraphs shorter than MIN_LEN_PARAG
book_parags_ls = [x for x in book_parags_ls if len(x) > MIN_LEN_PARAG]

# Trim any leading/trailing/multiple embedded whitespaces
book_parags_ls = [' '.join(x.split()) for x in book_parags_ls]

len(book_parags_ls)

2853

In [ ]:
# Agglomerate paragraphs into Documents of MIN_LEN_DOC=1000 chars

parag_ct = len(book_parags_ls)

doc_now_str = ''
doc_now_len = 0
docs = []

for i in range(parag_ct):
  # print(f'Processing Paragraph #{i}')
  parag_now_str = book_parags_ls[i]
  doc_now_str += parag_now_str
  doc_now_len += len(parag_now_str)
  if doc_now_len > MIN_LEN_DOC:
    docs.append(doc_now_str)
    doc_now_str = ''
    doc_now_len = 0

docs[-1] += doc_now_str

print(f'There are now {len(docs)} Documents of {MIN_LEN_DOC} chars or more')

There are now 594 Documents of 1000 chars or more


In [ ]:
# View the first 5 docs

docs[:5]

['The Project Gutenberg eBook of The Divine Comedy, by Dante AlighieriThis eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.Title: The Divine ComedyAuthor: Dante AlighieriTranslator: Henry Francis CaryRelease Date: August, 1997 [eBook #1008] [Most recently updated: July 4, 2022]Language: EnglishProduced by: Judith Smith and Natalie Salter*** START OF THE PROJECT GUTENBERG EBOOK THE DIVINE COMEDY ***THE DIVINE COMEDY OF DANTE ALIGHIERITranslated by THE REV. H. F. CARY, M.A.ContentsHELL CANTO I. CANTO II. CANTO III. CANTO IV. CANTO V. CANTO VI. CANTO VII. CANTO VIII. CANTO IX. CANTO X. CANTO XI. CANTO XII. 

In [ ]:
# View the first 500 chars in the 50th doc

docs[49][:500]

'A race I next espied, who held the head, And even all the bust above the stream. Midst these I many a face rememberd well. Thus shallow more and more the blood became, So that at last it but imbrud the feet; And there our passage lay athwart the foss.As ever on this side the boiling wave Thou seest diminishing, the Centaur said, So on the other, be thou well assurd, It lower still and lower sinks its bed, Till in that part it reuniting join, Where tis the lot of tyranny to mourn. There Heavns st'

In [ ]:
# View the last 100 chars in the last doc

docs[-1][-100:]

' help produce our new eBooks, and how to subscribe to our email newsletter to hear about new eBooks.'

## Pre-process and vectorize the documents

As part of preprocessing, we will:

* Expand Contractions
* Tokenize (split the documents into tokens)
* Define stopwords and filter out
* Lemmatize the tokens.
* Compute bigrams.
* Compute a bag-of-words representation of the data.

First we tokenize the text using a regular expression tokenizer from NLTK. We
remove numeric tokens and tokens that are only a single character, as they
don't tend to be useful, and the dataset contains a lot of them.

.. Important::

   This tutorial uses the nltk library for preprocessing, although you can
   replace it with something else if you want.




### Expand Contractions

In [ ]:
# Expand Contractions (e.g. can't -> can not)

import contractions

# Test
print('Not fixed:')
contractions.fix("yall're happy now", slang=False) # default: true
print('\n\nFixed:')
contractions.fix("you're happy now", slang=False) # default: true

Not fixed:


"yall're happy now"



Fixed:


'you are happy now'

In [ ]:
# Expand all Contractions paragraph by paragraph
docs_clean_ls = []
for adoc in docs:
  docs_clean_ls.append(contractions.fix(adoc))

docs = docs_clean_ls
docs[:5]

['The Project Gutenberg eBook of The Divine Comedy, by Dante AlighieriThis eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.Title: The Divine ComedyAuthor: Dante AlighieriTranslator: Henry Francis CaryRelease Date: August, 1997 [eBook #1008] [Most recently updated: July 4, 2022]Language: EnglishProduced by: Judith Smith and Natalie Salter*** START OF THE PROJECT GUTENBERG EBOOK THE DIVINE COMEDY ***THE DIVINE COMEDY OF DANTE ALIGHIERITranslated by THE REV. H. F. CARY, M.A.ContentsHELL CANTO I. CANTO II. CANTO III. CANTO IV. CANTO V. CANTO VI. CANTO VII. CANTO VIII. CANTO IX. CANTO X. CANTO XI. CANTO XII. 

### Tokenize Text

In [ ]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [ ]:
# Verify docs

# content and token count of first doc
docs[0]
print(f'\n\nThere are {len(docs[0])} tokens in the first document')

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'the',
 'divine',
 'comedy',
 'by',
 'dante',
 'alighierithis',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'in',
 'the',
 'united',
 'states',
 'and',
 'most',
 'other',
 'parts',
 'of',
 'the',
 'world',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever',
 'you',
 'may',
 'copy',
 'it',
 'give',
 'it',
 'away',
 'or',
 're',
 'use',
 'it',
 'under',
 'the',
 'terms',
 'of',
 'the',
 'project',
 'gutenberg',
 'license',
 'included',
 'with',
 'this',
 'ebook',
 'or',
 'online',
 'at',
 'www',
 'gutenberg',
 'org',
 'if',
 'you',
 'are',
 'not',
 'located',
 'in',
 'the',
 'united',
 'states',
 'you',
 'will',
 'have',
 'to',
 'check',
 'the',
 'laws',
 'of',
 'the',
 'country',
 'where',
 'you',
 'are',
 'located',
 'before',
 'using',
 'this',
 'ebook',
 'title',
 'the',
 'divine',
 'comedyauthor',
 'dante',
 'alighieritranslator',
 'henry',
 'francis',
 'caryrelease



There are 207 tokens in the first document


### Customize Stopwords

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stopwords_ls = stopwords.words('english')

print(f'\nThe first ten stopwords:')
stopwords_ls[20:]
print(f'\n\nThere are [{len(stopwords_ls)}] English stopwords imported from NLTK')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True


The first ten stopwords:


['himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 "don



There are [179] English stopwords imported from NLTK


In [ ]:
# Remove all stopwords with contractions (optional - could also remove fragments like 'll', 're', and 've' but not necessary)

stopwords_ls = [x for x in stopwords_ls if not "'" in x]
stopwords_ls[20:]
print(f'\n\nThere are now {len(stopwords_ls)} stopwords after removing words with contractions')

['herself',
 'it',
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each',
 'few',
 'more',
 'most',
 'other',
 'some',
 'such',
 'no',
 'nor',
 'not',
 'only',
 'own',
 'same',
 'so',
 'than',
 'too',
 'very',
 's',
 't',
 'can',
 'will',
 'just',
 'don',
 'should',
 'now',
 'd',
 'll',
 'm',
 'o',
 're',
 've',
 'y',
 'ain',
 



There are now 153 stopwords after removing words with contractions


**[CUSTOMIZE] Stopwords list by adding or deleting tokens**

In [ ]:
# [CUSTOMIZE] Stopwords to ADD or DELETE from default NLTK English stopword list

STOPWORDS_ADD_SET = set(['bazinga', 'woo-hoo']) # Edit this list to add new words to the stopwords list
STOPWORDS_DEL_SET = set(['the']) # Edit this list to remove exising words from the stopwords list

stopwords_en_ls = list(set(stopwords_ls).difference(set(STOPWORDS_DEL_SET)).union(set(STOPWORDS_ADD_SET)))
print(f'Final Count after Customized Add/Del: {len(stopwords_en_ls)} Stopwords')

Final Count after Customized Add/Del: 154 Stopwords


### Lemmatize

We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a
stemmer in this case because it produces more readable words. Output that is
easy to read is very desirable in topic modelling.




In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
%%time

# NOTE: 0m24s @03:48 on 20220228 Colab Pro 

# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

CPU times: user 4 s, sys: 80.4 ms, total: 4.08 s
Wall time: 4.13 s


We find bigrams in the documents. Bigrams are sets of two adjacent words.
Using bigrams we can get phrases like "machine_learning" in our output
(spaces are replaced with underscores); without bigrams we would only get
"machine" and "learning".

Note that in the code below, we find bigrams and then add them to the
original data, because we would like to keep the words "machine" and
"learning" as well as the bigram "machine_learning".

.. Important::
    Computing n-grams of large dataset can be very computationally
    and memory intensive.




### Identify Bi- and Tri-Grams

In [ ]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

We remove rare words and common words based on their *document frequency*.
Below we remove words that appear in less than 20 documents or in more than
50% of the documents. Consider trying to remove words only based on their
frequency, or maybe combining that with this approach.




In [ ]:
#@markdown Minimum number of documents a word must appear in: (default 20)

Min_No_Documents = 20 #@param {type:"slider", min:5, max:100, step:1}

#@markdown Max percent of documents a word can appear in: (default 0.50 or 50%)

Max_Percent_Documents = 0.5 #@param {type:"slider", min:0.05, max:0.9, step:0.01}


In [ ]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=Min_No_Documents, no_above=Max_Percent_Documents)

Finally, we transform the documents to a vectorized form. We simply compute
the frequency of each word, including the bigrams.




In [ ]:
# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

Let's see how many tokens and documents we have to train on.




In [ ]:
# Corpus as a list of documents, each a list of tokens identifyed by dictionary tuples

len(corpus)

594

In [ ]:
type(corpus[0])

list

In [ ]:
corpus[0][:30]

[(0, 2),
 (1, 1),
 (2, 1),
 (3, 34),
 (4, 1),
 (5, 3),
 (6, 4),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 1),
 (16, 1),
 (17, 1),
 (18, 2),
 (19, 2),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 2),
 (25, 1),
 (26, 1),
 (27, 2),
 (28, 1),
 (29, 1)]

In [ ]:
print('Number of unique tokens: %d' % len(dictionary))  # Orig 1864
print('Number of documents: %d' % len(corpus))          # Orig 1740

Number of unique tokens: 789
Number of documents: 594


## Training

We are ready to train the LDA model. We will first discuss how to set some of
the training parameters.

First of all, the elephant in the room: how many topics do I need? There is
really no easy answer for this, it will depend on both your data and your
application. I have used 10 topics here because I wanted to have a few topics
that I could interpret and "label", and because that turned out to give me
reasonably good results. You might not need to interpret all your topics, so
you could use a large number of topics, for example 100.

``chunksize`` controls how many documents are processed at a time in the
training algorithm. Increasing chunksize will speed up training, at least as
long as the chunk of documents easily fit into memory. I've set ``chunksize =
2000``, which is more than the amount of documents, so I process all the
data in one go. Chunksize can however influence the quality of the model, as
discussed in Hoffman and co-authors [2], but the difference was not
substantial in this case.

``passes`` controls how often we train the model on the entire corpus.
Another word for passes might be "epochs". ``iterations`` is somewhat
technical, but essentially it controls how often we repeat a particular loop
over each document. It is important to set the number of "passes" and
"iterations" high enough.

I suggest the following way to choose iterations and passes. First, enable
logging (as described in many Gensim tutorials), and set ``eval_every = 1``
in ``LdaModel``. When training the model look for a line in the log that
looks something like this::

   2016-06-21 15:40:06,753 - gensim.models.ldamodel - DEBUG - 68/1566 documents converged within 400 iterations

If you set ``passes = 20`` you will see this line 20 times. Make sure that by
the final passes, most of the documents have converged. So you want to choose
both passes and iterations to be high enough for this to happen.

We set ``alpha = 'auto'`` and ``eta = 'auto'``. Again this is somewhat
technical, but essentially we are automatically learning two parameters in
the model that we usually would have to specify explicitly.




In [ ]:
#@markdown How many Topics do you want to find?

No_of_Topics = 50 #@param {type:"slider", min:2, max:200, step:1}

#@markdown Default 10-50 depending upon how large the text and diverse the vocabulary

In [ ]:
%%time

# NOTE: 2m01s @20220328 with Google Colab/CPU on Harry Potter

# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = No_of_Topics
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

CPU times: user 25.9 s, sys: 10 s, total: 35.9 s
Wall time: 26.1 s


We can compute the topic coherence of each topic. Below we display the
average topic coherence and print the topics in order of topic coherence.

Note that we use the "Umass" topic coherence measure here (see
:py:func:`gensim.models.ldamodel.LdaModel.top_topics`), Gensim has recently
obtained an implementation of the "AKSW" topic coherence measure (see
accompanying blog post, http://rare-technologies.com/what-is-topic-coherence/).

If you are familiar with the subject of the articles in this dataset, you can
see that the topics below make a lot of sense. However, they are not without
flaws. We can see that there is substantial overlap between some topics,
others are hard to interpret, and most of them have at least some terms that
seem out of place. If you were able to do better, feel free to share your
methods on the blog at http://rare-technologies.com/lda-training-tips/ !




In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.

from pprint import pprint
pprint(top_topics)

print('\n\n')
for i,atopic in enumerate(top_topics):
  print(f'Topic #{i}: coherence = {atopic[1]}')

avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('\nAverage topic coherence: %.4f.' % avg_topic_coherence)

[([(0.01577849, 'we'),
   (0.01274178, 'thus'),
   (0.012697416, 'u'),
   (0.01143468, 'then'),
   (0.0113363275, 'their'),
   (0.011104351, 'now'),
   (0.011092624, 'they'),
   (0.0104272105, 'had'),
   (0.0091726165, 'if'),
   (0.009044637, 'there'),
   (0.008894514, 'them'),
   (0.008179583, 'eye'),
   (0.008050705, 'thy'),
   (0.0074181356, 'were'),
   (0.0072661242, 'what'),
   (0.0070694126, 'more'),
   (0.0067032594, 'our'),
   (0.006676699, 'guide'),
   (0.0064285076, 'where'),
   (0.0063811536, 'thee')],
  -0.7350012601168229),
 ([(0.013058011, 'thy'),
   (0.011748731, 'her'),
   (0.01151744, 'thee'),
   (0.010946495, 'such'),
   (0.010792032, 'their'),
   (0.009857129, 'more'),
   (0.009810686, 'thus'),
   (0.009282578, 'if'),
   (0.007998415, 'will'),
   (0.007978359, 'now'),
   (0.0079752775, 'she'),
   (0.007954601, 'no'),
   (0.007728981, 'or'),
   (0.0076973233, 'what'),
   (0.007637288, 'love'),
   (0.0074799885, 'then'),
   (0.0074750273, 'they'),
   (0.0074195913, 'we

# Visualize

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [ ]:
# vis_data = gensimvis.prepare(lda, corpus, dictionary)
vis_data = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis_data)

## Things to experiment with

* ``no_above`` and ``no_below`` parameters in ``filter_extremes`` method.
* Adding trigrams or even higher order n-grams.
* Consider whether using a hold-out set or cross-validation is the way to go for you.
* Try other datasets.

## Where to go from here

* Check out a RaRe blog post on the AKSW topic coherence measure (http://rare-technologies.com/what-is-topic-coherence/).
* pyLDAvis (https://pyldavis.readthedocs.io/en/latest/index.html).
* Read some more Gensim tutorials (https://github.com/RaRe-Technologies/gensim/blob/develop/tutorials.md#tutorials).
* If you haven't already, read [1] and [2] (see references).

## References

1. "Latent Dirichlet Allocation", Blei et al. 2003.
2. "Online Learning for Latent Dirichlet Allocation", Hoffman et al. 2010.


